In [109]:
import pymupdf
import pandas as pd
import re

# Open some document, for example a PDF (could also be EPUB, XPS, etc.)
doc = pymupdf.open("./docs/nnpl.pdf")

tables = []

# skip first table of first page
skip = False
for page in doc:
    for table in page.find_tables():
        if not skip:
            skip = True
            continue

        # columns name is first row
        df = pd.DataFrame(table.extract())
        df.columns = df.iloc[0]
        df = df[1:]

        # remove columns with name None
        df = df.loc[:, df.columns.notnull()]

        tables.append(df)

# merge all tables
df = pd.concat(tables)

print(df)

0    STT                                   Nội dung câu hỏi  \
1     1.  Nội dung nào sau đây không phải là đặc\nđiểm c...   
2   None                                               None   
3   None                                               None   
4   None                                               None   
5   None                                               None   
..   ...                                                ...   
17  None                                               None   
18  197.                      Phương án nào sau đây là sai:   
19  198.  Loại chế tài nào sau đây không phải là\nhình p...   
20  199.                    Khẳng định nào sau đây là đúng:   
21  200.  Hành vi gây thiệt hại trong tình thế cấp thiết...   

0                                           Phương án Đáp\nán  Phần Câṕ\nđộ  
1   A. Nhà nước ban hành pháp luật\nB. Nhà nước qu...       D     1        1  
2                                                None    None  None     None  
3     

In [110]:
DUMMDUMMY_QUESTION = {
    "question": "",
    "answers": "",
    "correct_answer": None,
    "level": "",
    "part": "",
}

In [111]:
from copy import deepcopy

In [112]:
# replace \n with space
df.columns = df.columns.str.replace("\n", " ")

print(df.columns)

Index(['STT', 'Nội dung câu hỏi', 'Phương án', 'Đáp án', 'Phần', 'Câṕ độ'], dtype='object', name=0)


In [113]:
current_question = None
questions = []

for idx, row in df.iterrows():
    if row["STT"]:
        if current_question:
            questions.append(current_question)
        current_question = deepcopy(DUMMDUMMY_QUESTION)

    if row["Nội dung câu hỏi"]:
        current_question["question"] += " " + row["Nội dung câu hỏi"]

    if row["Phương án"]:
        current_question["answers"] += row["Phương án"]

    if row["Đáp án"]:
        current_question["correct_answer"] = ["A", "B", "C", "D"].index(
            row["Đáp án"][0]
        )

    if row["Phần"]:
        current_question["part"] = row["Phần"]

    if row["Câṕ độ"]:
        current_question["level"] = row["Câṕ độ"]

questions.append(current_question)

In [114]:
# conver "A. ajsadjhsd \n B. asdjasd \n C. asdjasd \n D. asdjasd" to ["ajsadjhsd", "asdjasd", "asdjasd", "asdjasd"]
for question in questions:
    # replace \n with space
    question["question"] = question["question"].replace("\n", " ")
    question["answers"] = question["answers"].replace("\n", " ")

    # split answers
    question["answers"] = re.split(r"\s*[A-D]\.\s*", question["answers"])[1:]

    question["question"] += f" (phần {question['part']}, cấp độ {question['level']})"
    question.pop("part")
    question.pop("level")

print(len(questions))

200


In [115]:
import json

with open("data/nnpl.json", "w") as f:
    json.dump(questions, f, indent=4, ensure_ascii=False)